In [ ]:
from simplekml import (Kml, OverlayXY, ScreenXY, Units, RotationXY,
                       AltitudeMode, Camera)
import numpy as np
import matplotlib.pyplot as plt

##### make_kml function
def make_kml(llcrnrlon, llcrnrlat, urcrnrlon, urcrnrlat,
             figs, colorbar=None, **kw):
    """TODO: LatLon bbox, list of figs, optional colorbar figure,
    and several simplekml kw..."""

    kml = Kml()
    altitude = kw.pop('altitude', 2e7)
    roll = kw.pop('roll', 0)
    tilt = kw.pop('tilt', 0)
    altitudemode = kw.pop('altitudemode', AltitudeMode.relativetoground)
    camera = Camera(latitude=np.mean([urcrnrlat, llcrnrlat]),
                    longitude=np.mean([urcrnrlon, llcrnrlon]),
                    altitude=altitude, roll=roll, tilt=tilt,
                    altitudemode=altitudemode)

    kml.document.camera = camera
    draworder = 0
    for fig in figs:  # NOTE: Overlays are limited to the same bbox.
        draworder += 1
        ground = kml.newgroundoverlay(name='GroundOverlay')
        ground.draworder = draworder
        ground.visibility = kw.pop('visibility', 1)
        ground.name = kw.pop('name', 'overlay')
        ground.color = kw.pop('color', '9effffff')
        ground.atomauthor = kw.pop('author', 'ocefpaf')
        ground.latlonbox.rotation = kw.pop('rotation', 0)
        ground.description = kw.pop('description', 'Matplotlib figure')
        ground.gxaltitudemode = kw.pop('gxaltitudemode',
                                       'clampToSeaFloor')
        ground.icon.href = fig
        ground.latlonbox.east = llcrnrlon
        ground.latlonbox.south = llcrnrlat
        ground.latlonbox.north = urcrnrlat
        ground.latlonbox.west = urcrnrlon

    if colorbar:  # Options for colorbar are hard-coded (to avoid a big mess).
        screen = kml.newscreenoverlay(name='ScreenOverlay')
        screen.icon.href = colorbar
        screen.overlayxy = OverlayXY(x=0, y=0,
                                     xunits=Units.fraction,
                                     yunits=Units.fraction)
        screen.screenxy = ScreenXY(x=0.015, y=0.075,
                                   xunits=Units.fraction,
                                   yunits=Units.fraction)
        screen.rotationXY = RotationXY(x=0.5, y=0.5,
                                       xunits=Units.fraction,
                                       yunits=Units.fraction)
        screen.size.x = 0
        screen.size.y = 0
        screen.size.xunits = Units.fraction
        screen.size.yunits = Units.fraction
        screen.visibility = 1

    kmzfile = kw.pop('kmzfile', 'overlay.kmz')
    kml.savekmz(kmzfile)
#####

##### gearth_fig function
def gearth_fig(llcrnrlon, llcrnrlat, urcrnrlon, urcrnrlat, pixels=1024):
    """Return a Matplotlib `fig` and `ax` handles for a Google-Earth Image."""
    aspect = np.cos(np.mean([llcrnrlat, urcrnrlat]) * np.pi/180.0)
    xsize = np.ptp([urcrnrlon, llcrnrlon]) * aspect
    ysize = np.ptp([urcrnrlat, llcrnrlat])
    aspect = ysize / xsize

    if aspect > 1.0:
        figsize = (10.0 / aspect, 10.0)
    else:
        figsize = (10.0, 10.0 * aspect)

    if False:
        plt.ioff()  # Make `True` to prevent the KML components from poping-up.
    fig = plt.figure(figsize=figsize,
                     frameon=False,
                     dpi=pixels//10)
    # KML friendly image.  If using basemap try: `fix_aspect=False`.
    ax = fig.add_axes([0, 0, 1, 1])
    ax.set_xlim(llcrnrlon, urcrnrlon)
    ax.set_ylim(llcrnrlat, urcrnrlat)
    return fig, ax
#####

In [ ]:
# Modules for figure creation

import pandas as pd
import os
import xarray as xr
import geopandas as gpd
from datetime import datetime, timedelta
import cartopy.crs as ccrs
import matplotlib.colors as colors

In [ ]:
# Establish directories

def create_missing_directories():
    # Define the path to the parent directory
    parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

    # Check if 'data' folder exists in the parent directory
    data_dir = os.path.join(parent_dir, 'data')
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
        print("'data' folder created in the parent directory.")
    else:
        print("'data' folder already exists in the parent directory.")

    # Check if 'kmz' directory exists inside 'data' folder
    kmz_dir = os.path.join(data_dir, 'kmz')
    if not os.path.exists(kmz_dir):
        os.makedirs(kmz_dir)
        print("'KMZ' directory created inside 'data' folder.")
    else:
        print("'KMZ' directory already exists inside 'data' folder.")

if __name__ == "__main__":
    create_missing_directories()

# Establish directory locations

parent_dir    = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_dir      = os.path.join(parent_dir, 'data')
satellite_dir = os.path.join(data_dir, 'satellite')
output_dir    = os.path.join(parent_dir, 'Output/kml')
NEODASS_dir   = os.path.join(satellite_dir, 'NEODASS')
floats_dir    = os.path.join(parent_dir, 'Data/Floats')
kmz_dir       = os.path.join(parent_dir, 'Data/kmz')

In [ ]:
# Plotting preferences

# Global
min_lon = -35 #-35
max_lon = -5 #-5
min_lat = 55 #55
max_lat = 66 #66

# Float colors
# For additional floats, add the name (as it appears in the 'Float_positions.csv') and color you'd like (as a hexcode)
float_colors = {
        '4903532':  '#B4184C',
        'navis102': '#F5A300',
        '1902637':  '#0000E0',
        'navis101': '#FBFF1F'}

# Define unique symbols for different object types
object_symbols = {
    'glider': 'o',
    'Float': 's',
    'Ship'  : '^'
    # Add more object types and their symbols here
}

# DAC
skip_dac             = False
DAC_color            = 'spring'
DAC_plot_as_log      = False # Default False
DAC_plot_lim_max     = 0.5   # Default 5
DAC_plot_lim_min     = 0   # Default 0

# Surface Currents
skip_ssc             = False
SSC_color            = 'spring'
SSC_plot_as_log      = False # Default False
SSC_plot_lim_max     = 0.5   # Default 5
SSC_plot_lim_min     = 0   # Default 0

# 1000m currents
skip_dwc             = False
DWC_color            = 'spring'
DWC_plot_as_log      = False
DWC_plot_lim_max     = 0.5
DWC_plot_lim_min     = 0

# CHLA
skip_chla            = False
CHLA_color           = 'YlGnBu_r'
CHLA_plot_as_log     = True # Default True
CHLA_plot_lim_max    = 10   # Default 10
CHLA_plot_lim_min    = 0.1  # Default 0.1

In [ ]:

### Plotting most recent data from CMEMS
## List most recent files

# List to store matching file names
matching_files = []

# Loop through the files in the directory
for filename in os.listdir(satellite_dir):
    if filename.endswith('.nc'):
        matching_files.append(filename)

print(matching_files)

## List variables from these files
# Dictionary to store files by their base names (excluding '_a_' or '_b_') and key variables
file_dict = {}

for file in matching_files:
    # Identify the base name by removing '_a' or '_b' if present
    base_name = file
    if base_name not in file_dict:
        file_dict[base_name] = {}
    filepath = os.path.join(satellite_dir, file)
    ds = xr.open_dataset(filepath)
    variables = list(ds.data_vars.keys())

    print(file)
    print(variables)

    key_variables = [var for var in variables if 'longitude' not in var.lower() and 'latitude' not in var.lower()]

    if key_variables:
        for key_variable in key_variables:
            if file not in file_dict[base_name]:
                file_dict[base_name][file] = {}
            file_dict[base_name][file][key_variable] = {
                'variable': key_variable,
            }
    else:
        print(f"No suitable key variable found in file: {file}")

    ds.close()

# Print the results
for base_name, files in file_dict.items():
    for file, key_variable in files.items():
        print(f'File: {file}')
        print(f'Base Name: {base_name}')
        print(f'Key Variable: {key_variable}')

    filepath  = os.path.join(satellite_dir, file)
    dataset = xr.open_dataset(filepath)

    # Print global attributes
    print("Global attributes:")
    for attr_name, attr_value in dataset.attrs.items():
        print(f"  {attr_name}: {attr_value}")

    # Print variable attributes
    print("\nVariable attributes:")
    for var_name, var in dataset.variables.items():
        print(f"Variable: {var_name}")
        for attr_name, attr_value in var.attrs.items():
            print(f"  {attr_name}: {attr_value}")

# Initialize var_dict with categories as keys and empty dictionaries as values
var_dict = {'chl': {}, 'temp': {}, 'sla': {}, 'adt': {}, 'bbp': {}, 'dissic': {}, 'ugos':{}, 'uo':{}, 'dwc':{}} 

for base_name, files in file_dict.items():
    for file, variables in files.items():
        contains_adt = False
        contains_ugos = False
        contains_uo = False
        contains_dwc = False
        
        for key_variable, details in variables.items():
            data_var = details['variable']
            var_name_lower = data_var.lower()
            
            if 'chl' in var_name_lower and var_name_lower == 'chl':
                var_dict['chl'][file] = data_var
            elif 'temp' in var_name_lower:
                var_dict['temp'][file] = data_var
            elif 'sla' in var_name_lower:
                var_dict['sla'][file] = data_var
            elif 'adt' in var_name_lower:
                var_dict['adt'][file] = data_var
                contains_adt = True
            elif 'bbp' in var_name_lower and var_name_lower == 'bbp':
                var_dict['bbp'][file] = data_var
            elif 'dissic' in var_name_lower and var_name_lower == 'dissic':
                var_dict['dissic'][file] = data_var
            elif 'ugos' in var_name_lower and var_name_lower == 'ugos':
                contains_ugos = True
            elif 'uo' in var_name_lower and var_name_lower == 'uo' and 'averaged' in base_name:
                contains_uo = True
            elif 'uo' in var_name_lower and var_name_lower == 'uo' and '1000m' in base_name:
                contains_dwc = True
                # Delay adding 'ugos' to var_dict until after the adt check

        # After checking all variables in the file
        if contains_ugos:
            if contains_adt:
                # Add to 'adt' category if the file contains both 'adt' and 'ugos'
                var_dict['adt'][file] = data_var
            else:
                # Add to 'ugos' category if the file contains only 'ugos'
                var_dict['ugos'][file] = data_var
        if contains_uo:
            var_dict['uo'][file] = data_var
        if contains_dwc:
            var_dict['dwc'][file] = data_var

# We load shape files for bathymetry lines
#first_line_path = 'c:\\Users\\flapet\\OneDrive - NOC\\Documents\\NRT_viz\\biocarbon_nrt_data_viz/Data/ne_10m_bathymetry_all/ne_10m_bathymetry_J_1000.shp'
#second_line_path = 'c:\\Users\\flapet\\OneDrive - NOC\\Documents\\NRT_viz\\biocarbon_nrt_data_viz/Data/ne_10m_bathymetry_all/ne_10m_bathymetry_I_2000.shp'

first_line_path  = 'C:\\Users\\hanshil\\Documents\\GitHub\\biocarbon_nrt_data_viz\\data\\bathymetry\\ne_10m_bathymetry_J_1000.shp'
second_line_path = 'C:\\Users\\hanshil\\Documents\\GitHub\\biocarbon_nrt_data_viz\\data\\bathymetry\\ne_10m_bathymetry_I_2000.shp'

gdf_1000 = gpd.read_file(first_line_path)
gdf_2000 = gpd.read_file(second_line_path)

In [ ]:
# Load autonomy positions
csv_file = 'C:\\Users\\hanshil\\Documents\\GitHub\\biocarbon_nrt_data_viz\\Plotting_tools\\shared_data\\rt_positions.csv'  # Update this path
df = pd.read_csv(csv_file)
df['date'] = pd.to_datetime(df['date']) # Convert the 'time' column to datetime
xx_day_offset = [-1, -2, -3, -4]

for var_category, files in var_dict.items():
    if var_category in ['adt', 'dwc', 'uo']:  
        for day_offset in xx_day_offset:  # Plot the last day # Change this silly bit of code to be something more readable
            if len(files) > 1:
                data_vars = []
                for file, data_var in files.items():
                    cur_data_var = xr.open_dataset(os.path.join(satellite_dir, file))
                    data_vars.append(cur_data_var[data_var].isel(time=day_offset))
                aligned_data_vars = xr.align(*data_vars, join='outer')
                combined_data_var = aligned_data_vars[0]
                for var in aligned_data_vars[1:]:
                    combined_data_var = combined_data_var.combine_first(var)
                combined_data_var_mean = np.nanmean(combined_data_var, axis=0)
                date_of_plot = str(combined_data_var['time'].data[0])[0:10]
            elif len(files) == 1:
                file, data_var = next(iter(files.items()))
                cur_data_var = xr.open_dataset(os.path.join(satellite_dir, file))
                date_of_plot = str(cur_data_var['time'].data[day_offset])[0:10]
                combined_data_var_mean = cur_data_var[data_var].isel(time=day_offset).data
            else:
                continue

            xx_plot_min = DAC_plot_lim_min
            xx_plot_max = DAC_plot_lim_max
            log_scaling = DAC_plot_as_log
            norm = colors.LogNorm(vmin=xx_plot_min, vmax=xx_plot_max) if log_scaling else colors.Normalize(vmin=xx_plot_min, vmax=xx_plot_max)

            # Units for colorbar
            xx_plot_units = cur_data_var[data_var].attrs.get('units')
            xx_plot_cbar_label = 'Sea surface velocity (' + xx_plot_units + ')'

            # Clip the data within the specified range
            data_to_plot_1day = combined_data_var_mean

            # If the first dimension of the data is 1, remove it
            if data_to_plot_1day.shape[0] == 1:
                data_to_plot_1day = data_to_plot_1day[0]

            # Extract longitudes and latitudes
            longitudes = cur_data_var['longitude'].data
            latitudes = cur_data_var['latitude'].data

            if var_category in ['uo','dwc']:
                ugos = cur_data_var['uo'].isel(time=day_offset).data
                vgos = cur_data_var['vo'].isel(time=day_offset).data
            if var_category == 'adt':
                ugos = cur_data_var['ugos'].isel(time=day_offset).data
                vgos = cur_data_var['vgos'].isel(time=day_offset).data

            # Extracting latest lead float position
            date_of_plot_filter = datetime.strptime(date_of_plot, '%Y-%m-%d')
            df_filtered = df[df['date'] <= date_of_plot_filter]

            legend_handles = []

            # Process each unique object type
            for obj_type, symbol in object_symbols.items():
                if obj_type == 'Float':
                    type_data = df_filtered[df_filtered['platform_type'] == obj_type]
                    unique_names = type_data['platform_id'].unique()
                    obj_data = type_data.sort_values(by='date', ascending=False)

            # Define the bounds of the 2-degree square around the center
            lon_min = obj_data['lon'].iloc[0] - 3
            lon_max = obj_data['lon'].iloc[0] + 3
            lat_min = obj_data['lat'].iloc[0] - 2
            lat_max = obj_data['lat'].iloc[0] + 2

            # Find the indices of the longitudes and latitudes within the bounds
            lon_indices = np.where((longitudes >= lon_min) & (longitudes <= lon_max))[0]
            lat_indices = np.where((latitudes >= lat_min) & (latitudes <= lat_max))[0]

            # Extract the subset of data within the bounds
            subsampled_longitudes = longitudes[lon_indices]
            subsampled_latitudes = latitudes[lat_indices]
            subsampled_ugos = ugos[np.ix_(lat_indices, lon_indices)]
            subsampled_vgos = vgos[np.ix_(lat_indices, lon_indices)]

            # Extract the lon and lat from the dataset only once
            x = subsampled_longitudes
            y = subsampled_latitudes

            # From the U and V vector compute the speed, we use it as our colour map
            u = subsampled_ugos
            v = subsampled_vgos

            #time_step = 6 * 3600  # Convert 6 hours to seconds

            #U_km_per_6hr = u * time_step / 1000  # Convert from m/s to km/6hr
            #V_km_per_6hr = v * time_step / 1000  # Convert from m/s to km/6hr
            speed = np.sqrt(u**2 + v**2)

            # Plot the current vectors field and the coastline
            png_filename = 'quiver_plot.png'
            pixels = 1024 * 10
            fig, ax = gearth_fig(llcrnrlon=lon_min,
                                 llcrnrlat=lat_min,
                                 urcrnrlon=lon_max,
                                 urcrnrlat=lat_max,
                                 pixels=pixels)
            im = ax.quiver(x, y, u, v,
                           speed, angles='xy', scale_units='xy', 
                           cmap='spring', width=0.002, scale = 2,
                           norm=norm) #scale = 0.00025)
            ax.quiverkey(im, 0.86, 0.45, 0.2, "0.2 m s^{-1}$", labelpos='W')
            ax.set_axis_off()
            fig.savefig('overlay1.png', transparent=False, format='png')
            #plt.savefig(png_filename, bbox_inches='tight', pad_inches=0)
            #plt.close()

            fig = plt.figure(figsize=(1.0, 4.0), facecolor=None, frameon=False)
            ax = fig.add_axes([0.0, 0.05, 0.2, 0.9])
            cb = fig.colorbar(im, cax=ax)
            cb.set_label('Current velocity [m s^-1]', rotation=-90, color='k', labelpad=20)
            fig.savefig('legend.png', transparent=False, format='png')  # Change transparent to True if your colorbar is not on space :)
            
            plot_prefix = 'NONE'

            if var_category == 'adt':
                plot_prefix = 'SC_'
            elif var_category == 'uo':
                plot_prefix = 'DAC_'
            elif var_category == 'dwc':
                plot_prefix = 'DWC_'

            output_file_path = filepath = os.path.join(kmz_dir, plot_prefix + date_of_plot + '.kmz')

            make_kml(llcrnrlon=lon_min, llcrnrlat=lat_min,
                    urcrnrlon=lon_max, urcrnrlat=lat_max,
                    figs=['overlay1.png'], colorbar='legend.png',
                    kmzfile= output_file_path, name= plot_prefix + date_of_plot)
            print('Made plot for ' + plot_prefix + date_of_plot)

In [ ]:
# Load autonomy positions
csv_file = 'C:\\Users\\hanshil\\Documents\\GitHub\\biocarbon_nrt_data_viz\\Plotting_tools\\shared_data\\rt_positions.csv'  # Update this path
df = pd.read_csv(csv_file)
df['date'] = pd.to_datetime(df['date']) # Convert the 'time' column to datetime
xx_day_offset = [-1]

for var_category, files in var_dict.items():
    if var_category in ['chl']:  
        for day_offset in xx_day_offset:  # Plot the last day # Change this silly bit of code to be something more readable
            if len(files) > 1:
                data_vars = []
                for file, data_var in files.items():
                    cur_data_var = xr.open_dataset(os.path.join(satellite_dir, file))
                    data_vars.append(cur_data_var[data_var].isel(time=day_offset))
                aligned_data_vars = xr.align(*data_vars, join='outer')
                combined_data_var = aligned_data_vars[0]
                for var in aligned_data_vars[1:]:
                    combined_data_var = combined_data_var.combine_first(var)
                combined_data_var_mean = np.nanmean(combined_data_var, axis=0)
                date_of_plot = str(combined_data_var['time'].data[0])[0:10]
            elif len(files) == 1:
                file, data_var = next(iter(files.items()))
                cur_data_var = xr.open_dataset(os.path.join(satellite_dir, file))
                date_of_plot = str(cur_data_var['time'].data[day_offset])[0:10]
                combined_data_var_mean = cur_data_var[data_var].isel(time=day_offset).data
            else:
                continue

            xx_plot_min = CHLA_plot_lim_min
            xx_plot_max = CHLA_plot_lim_max
            log_scaling = CHLA_plot_as_log
            norm = colors.LogNorm(vmin=xx_plot_min, vmax=xx_plot_max) if log_scaling else colors.Normalize(vmin=xx_plot_min, vmax=xx_plot_max)

            # Units for colorbar
            xx_plot_units = cur_data_var[data_var].attrs.get('units')
            xx_plot_cbar_label = 'Sea surface velocity (' + xx_plot_units + ')'

            # Clip the data within the specified range
            data_to_plot_1day = combined_data_var_mean

            # If the first dimension of the data is 1, remove it
            if data_to_plot_1day.shape[0] == 1:
                data_to_plot_1day = data_to_plot_1day[0]

            # Extracting latest lead float position
            date_of_plot_filter = datetime.strptime(date_of_plot, '%Y-%m-%d')
            df_filtered = df[df['date'] <= date_of_plot_filter]

            legend_handles = []

            # Process each unique object type
            for obj_type, symbol in object_symbols.items():
                if obj_type == 'Float':
                    type_data = df_filtered[df_filtered['platform_type'] == obj_type]
                    unique_names = type_data['platform_id'].unique()
                    obj_data = type_data.sort_values(by='date', ascending=False)

            # Define the bounds of the 2-degree square around the center
            lon_min = obj_data['lon'].iloc[0] - 3
            lon_max = obj_data['lon'].iloc[0] + 3
            lat_min = obj_data['lat'].iloc[0] - 2
            lat_max = obj_data['lat'].iloc[0] + 2

            # Find the indices of the longitudes and latitudes within the bounds
            lon_indices = np.where((longitudes >= lon_min) & (longitudes <= lon_max))[0]
            lat_indices = np.where((latitudes >= lat_min) & (latitudes <= lat_max))[0]

            fig = plt.figure(figsize=(20, 10))
            ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mercator())
            ax.set_extent([min_lon, max_lon, min_lat, max_lat], crs=ccrs.PlateCarree())
            pixels = 1024 * 10
            
            fig, ax = gearth_fig(llcrnrlon=lon_min,
                                llcrnrlat=lat_min,
                                urcrnrlon=lon_max,
                                urcrnrlat=lat_max,
                                pixels=pixels)

            im = ax.pcolormesh(cur_data_var['longitude'].data, 
                               cur_data_var['latitude'].data, 
                               combined_data_var_mean, 
                               cmap=CHLA_color, 
                               norm=norm)
            ax.set_axis_off()
            fig.savefig('overlay1.png', transparent=False, format='png')

            fig = plt.figure(figsize=(1.0, 4.0), facecolor=None, frameon=False)
            ax = fig.add_axes([0.0, 0.05, 0.2, 0.9])
            cb = fig.colorbar(im, cax=ax)
            cb.set_label('Mean Dynamic Topography [m]', rotation=-90, color='k', labelpad=20)
            fig.savefig('legend.png', transparent=False, format='png')  # Change transparent to True if your colorbar is not on space :)
            
            plot_prefix = 'NONE'

            if var_category == 'chl':
                plot_prefix = 'CHL_'

            output_file_path = filepath = os.path.join(kmz_dir, plot_prefix + date_of_plot + '.kmz')

            make_kml(llcrnrlon=lon_min, llcrnrlat=lat_min,
                    urcrnrlon=lon_max, urcrnrlat=lat_max,
                    figs=['overlay1.png'], colorbar='legend.png',
                    kmzfile= output_file_path, name= plot_prefix + date_of_plot)
            print('Made plot for ' + plot_prefix + date_of_plot)